In [21]:
import jsonpickle

In [22]:
from project import Project
from typing import Dict
with open('projects.json') as f:
    projects : Dict[str,Project] = jsonpickle.decode(f.read())

In [23]:
projects.keys()
del projects['okhttp']
with open('okhttp.json') as o:
    projects['okhttp'] = jsonpickle.decode(o.read())['okhttp']
with open('commons-lang.json') as o:
    projects['commons-lang'] = jsonpickle.decode(o.read())['commons-lang']

In [24]:
projects.keys()

dict_keys(['retrofit', 'RxJava', 'zxing', 'okhttp', 'commons-lang'])

In [25]:
from contributor import Contributor
from typing import Collection

a : Collection[Contributor] = projects['okhttp'].methods[0].contributors

In [26]:
for c in a:
    print(c)

Jake Wharton<jw@squareup.com>: XP=0.058171079408606705


In [27]:
from statistics import mean, median, stdev, StatisticsError
with open("output.csv", "w") as f:
    f.write("Project,Package,Method,SLOC,Complexity,ModifyingCommits,Contributors,MaxExperienced,MinExperienced,MeanExperience,MedianExperience,StdevExperience\n")
for project_name, project in projects.items():
    for test in project.methods:
        experiences = [c.experience for c in test.contributors]
        with open("output.csv", "a") as f:
            contributors = len(experiences)
            if contributors == 0:
                print(f'{project_name}: {test.name}')
                max_xp = 'None'
                avg_xp = 'None'
                min_xp = 'None'
                median_xp = 'None'
            else:
                max_xp = max(experiences)
                avg_xp = mean(experiences)
                min_xp = min(experiences)
                median_xp = median(experiences)
            try:
                stdev_xp = stdev(experiences)
            except StatisticsError as err:
                stdev_xp = 'None'
            f.write(','.join([project_name, test.package, test.name, str(test.nloc), str(test.complexity), str(test.n_commits), str(contributors), str(max_xp), str(min_xp), str(avg_xp), str(median_xp), str(stdev_xp)]) + '\n')

retrofit: DefaultMethodsTest::user
retrofit: RequestFactoryTest::assertBody
RxJava: CompletableAndThenCompletableTest::andThenCompletableCompleteComplete
RxJava: CompletableAndThenCompletableTest::andThenCompletableCompleteError
RxJava: CompletableAndThenCompletableTest::andThenCompletableCompleteNever
RxJava: CompletableAndThenCompletableTest::andThenCompletableErrorComplete
RxJava: CompletableAndThenCompletableTest::andThenCompletableErrorNever
RxJava: CompletableAndThenCompletableTest::andThenCompletableErrorError
RxJava: CompletableAndThenCompletableTest::andThenCanceled
RxJava: CompletableAndThenCompletableTest::andThenFirstCancels
RxJava: CompletableAndThenCompletableTest::andThenSecondCancels
RxJava: CompletableAndThenCompletableTest::andThenDisposed
RxJava: CompletableAndThenCompletableTest::andThenNoInterrupt
RxJava: FlowableTakeTest2::accept
RxJava: FlowableTakeTest2::run
okhttp: ConnectionReuseTest::connectionsAreNotReusedWithRequestConnectionClose
okhttp: ConnectionReuseTes

In [28]:
with open("contributors.csv", "w") as f:
    f.write("Project,Package,Method,ModifyingCommits,ContributorsCount,ContributorsList\n")
for project_name, project in projects.items():
    for test in project.methods:
        contributors = [f"{c.name} <{c.email}> ({c.experience})" for c in test.contributors]
        with open("contributors.csv", "a") as f:
            count = len(contributors)
            f.write(','.join([project_name, test.package, test.name, str(test.n_commits), str(count), ','.join(contributors)]) + '\n')